In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [61]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_16164/COMPINT'
nx=128
ny=192
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Rc_32324/COMPINT'
nx=64
ny=96
ncomp=4


print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Rc_32324/COMPINT


In [62]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 64, 96, 4) , mean = 103.05342610677083 , std = 13.930115948965971 , Max = 165.0 , Min = 50.0 , type=float64
t_C_0 shape: (1, 64, 96) , mean = 101.63834635416667 , std = 14.385132330336374 , Max = 153.0 , Min = 50.0 , type=float64
t_C_1 shape: (1, 64, 96) , mean = 104.27815755208333 , std = 14.836575177972701 , Max = 165.0 , Min = 52.0 , type=float64
t_C_2 shape: (1, 64, 96) , mean = 102.767578125 , std = 14.420794930453319 , Max = 155.0 , Min = 56.0 , type=float64
t_C_3 shape: (1, 64, 96) , mean = 103.52962239583333 , std = 11.720337453062735 , Max = 154.0 , Min = 68.0 , type=float64


In [63]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (64, 96) , mean = -24.470377604166668 , std = 11.720337453062736 , Max = 26 , Min = -60 , type=int8


In [64]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen6144
size16156
size24229
ratio1= 1.001953125
ratio2= 0.688313802083


In [65]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 6144
mse = 0.0


In [66]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 64, 96, 4) , mean = 102.47249348958333 , std = 13.393139905073966 , Max = 163.0 , Min = 46.0 , type=float64
 channel0 shape: (64, 96) , mean = -26.830891927083332 , std = 14.472007916552482 , Max = 35 , Min = -82 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4570 Bytes
ratio1= 1.001953125
ratio2= 0.743815104167
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = -24.256022135416668 , std = 13.052043494599866 , Max = 31 , Min = -66 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4490 Bytes
ratio1= 1.001953125
ratio2= 0.730794270833
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -26.168782552083332 , std = 14.108362554192068 , Max = 22 , Min = -76 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4551 Bytes
ratio1= 1.001953125
ratio2= 0.74072265625
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = -24.854329427083332 , std = 11.596730193375032 , Max = 16 , Min = -79 , type=int8
   olen: 6144 Bytes
   si

dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -32.566569010416664 , std = 16.434503317594324 , Max = 31 , Min = -86 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4787 Bytes
ratio1= 1.001953125
ratio2= 0.779134114583
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = -30.627115885416668 , std = 11.957299260877212 , Max = 15 , Min = -82 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4461 Bytes
ratio1= 1.001953125
ratio2= 0.72607421875
dlen= 6144
mse = 0.0



9
  t_C shape: (1, 64, 96, 4) , mean = 92.39644368489583 , std = 9.6419680342748 , Max = 172.0 , Min = 31.0 , type=float64
 channel0 shape: (64, 96) , mean = -37.608723958333336 , std = 9.873099207191784 , Max = 26 , Min = -88 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4025 Bytes
ratio1= 1.001953125
ratio2= 0.655110677083
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = -34.2109375 , std = 9.787659516380499 , Max = 24 , Min = -93 , type=int8
   olen: 6144 B

dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = -26.302408854166668 , std = 11.504613858394158 , Max = 22 , Min = -68 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4370 Bytes
ratio1= 1.001953125
ratio2= 0.711263020833
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -28.000813802083332 , std = 11.28753140518966 , Max = 19 , Min = -61 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4318 Bytes
ratio1= 1.001953125
ratio2= 0.702799479167
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = -26.958821614583332 , std = 10.230454734769017 , Max = 8 , Min = -53 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4215 Bytes
ratio1= 1.001953125
ratio2= 0.68603515625
dlen= 6144
mse = 0.0



17
  t_C shape: (1, 64, 96, 4) , mean = 108.3939208984375 , std = 15.220307944221103 , Max = 166.0 , Min = 47.0 , type=float64
 channel0 shape: (64, 96) , mean = -20.36474609375 , std = 16.435784943024295 , Max = 28 , Min = -80 , type=int8
   olen

dlen= 6144
mse = 0.0



24
  t_C shape: (1, 64, 96, 4) , mean = 101.519775390625 , std = 13.641884352006103 , Max = 154.0 , Min = 35.0 , type=float64
 channel0 shape: (64, 96) , mean = -27.657389322916668 , std = 14.171761900761146 , Max = 10 , Min = -93 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4550 Bytes
ratio1= 1.001953125
ratio2= 0.740559895833
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = -25.97802734375 , std = 14.201798696390327 , Max = 18 , Min = -91 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4541 Bytes
ratio1= 1.001953125
ratio2= 0.739095052083
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -26.436197916666668 , std = 14.116326323710982 , Max = 26 , Min = -91 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4560 Bytes
ratio1= 1.001953125
ratio2= 0.7421875
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = -25.849283854166668 , std = 11.856032253212643 , Max = 5 , Min = -75 , type=int8
   olen: 61

In [67]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.001953125
   ratio_2= 0.7303466796875
 
2
   ratio_1= 1.001953125
   ratio_2= 0.5693766276041666
 
3
   ratio_1= 1.001953125
   ratio_2= 0.6763509114583334
 
4
   ratio_1= 1.001953125
   ratio_2= 0.7227376302083334
 
5
   ratio_1= 1.001953125
   ratio_2= 0.7583821614583334
 
6
   ratio_1= 1.001953125
   ratio_2= 0.7275390625
 
7
   ratio_1= 1.001953125
   ratio_2= 0.7151285807291666
 
8
   ratio_1= 1.001953125
   ratio_2= 0.7678629557291666
 
9
   ratio_1= 1.001953125
   ratio_2= 0.6522623697916666
 
10
   ratio_1= 1.001953125
   ratio_2= 0.6690673828125
 
11
   ratio_1= 1.001953125
   ratio_2= 0.702392578125
 
12
   ratio_1= 1.001953125
   ratio_2= 0.6610921223958334
 
13
   ratio_1= 1.001953125
   ratio_2= 0.7329915364583334
 
14
   ratio_1= 1.001953125
   ratio_2= 0.74560546875
 
15
   ratio_1= 1.001953125
   ratio_2= 0.7041422526041666
 
16
   ratio_1= 1.001953125
   ratio_2= 0.7018636067708334
 
17
   ratio_1= 1.001953125
   ratio_2= 0.7418619791666666
 
18
   rati